In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model 
import pickle
import pandas as pd
import numpy as np

In [85]:
## Load the trained model, scaler pickle,onehot
model = load_model('model.h5')

## load the encoder and scaler
with open('label_encoder_gender.pkl','rb') as file :
    label_encoder_gender=pickle.load(file)

with open('oneHot_encoder_geo.pkl','rb') as file :
    oneHot_encoder_geo=pickle.load(file)

with open('scaler.pkl','rb') as file :
    scaler=pickle.load(file)

In [88]:
input_data = [{
    'CreditScore':600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' :50000
},{
    'CreditScore':600,
    'Geography': 'Germany',
    'Gender': 'Female',
    'Age' : 40,
    'Tenure' : 2,
    'Balance' : 6000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' :5000
}]

In [89]:
df = pd.DataFrame(input_data)

In [90]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000
1,600,Germany,Female,40,2,6000,2,1,1,5000


In [91]:
df['Gender'] = label_encoder_gender.transform(df['Gender'])

In [92]:
geo_encoder = oneHot_encoder_geo.transform(df[['Geography']]).toarray()
geo_encoder_DF = pd.DataFrame(geo_encoder,columns=oneHot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_DF.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,1.0,0.0


In [93]:
input_data = pd.concat([df.reset_index(drop=True),geo_encoder_DF],axis=1)

In [94]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0
1,600,Germany,0,40,2,6000,2,1,1,5000,0.0,1.0,0.0


In [95]:
input_data = input_data.drop('Geography',axis=1)

In [96]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0
1,600,0,40,2,6000,2,1,1,5000,0.0,1.0,0.0


In [97]:
## Scaling the input Data
input_scaled = scaler.transform(input_data)
input_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184],
       [-0.47154541, -1.09997489,  0.09477172, -1.03940009, -1.15940786,
         0.80510537,  0.63367318,  0.95214374, -1.63125121, -1.02020406,
         1.73668197, -0.56349184]])

In [98]:
input_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184],
       [-0.47154541, -1.09997489,  0.09477172, -1.03940009, -1.15940786,
         0.80510537,  0.63367318,  0.95214374, -1.63125121, -1.02020406,
         1.73668197, -0.56349184]])

In [103]:
## Predict Churn
prediction = model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [110]:
prediction_proba = prediction[1][0]
prediction_proba

np.float32(0.055021472)

In [111]:
if prediction_proba > 0.5 :
    print('The Customer is likely to churn.')
else:
    print('The Customer is not likely to churn.')

The Customer is not likely to churn.
